In [3]:

from pydantic_ai import Agent, RunContext
from pydantic_ai.common_tools.tavily import tavily_search_tool
from pydantic_ai.messages import ModelMessage
from pydantic_ai.models.google import GoogleModel
from pydantic_ai.providers.google import GoogleProvider
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider
from dotenv import load_dotenv
from dataclasses import dataclass
from datetime import datetime
from pydantic import Field


import nest_asyncio
nest_asyncio.apply()


import os
from tavily import TavilyClient
load_dotenv()
google_api_key=os.getenv('google_api_key')
tavily_key=os.getenv('tavily_key')
pse=os.getenv('pse')
openai_api_key=os.getenv('openai_api_key')
tavily_client = TavilyClient(api_key=tavily_key)
composio_api_key=os.getenv('composio_api_key')
# configure logfire
# import logfire
# logfire.configure(token=os.getenv('logfire_token'))
# logfire.instrument_pydantic_ai()


In [4]:

api_keys={
    'google_api_key':google_api_key,
    'tavily_key':tavily_key,
    'pse':pse,
    'openai_api_key':openai_api_key,
    'composio_key':composio_api_key
}


In [1]:
from t_ai.t_agent import TAgent
from t_ai.utils.helper_functions import MCP_server_helper
from t_ai.PrebuiltTools.google_tools import search_images_tool

In [4]:
from t_ai.utils.helper_functions import MCP_server_helper
mcp_server_helper=MCP_server_helper()
mcp_server_helper.add_mcp_server(type='http', mcp_server_url=os.getenv('gmail_mcp_server'), headers=None)
servers=mcp_server_helper.get_mcp_servers()

In [5]:
# tools=[search_images_tool(api_key=api_keys['google_api_key'],search_engine_id=api_keys['pse']),code_execution_tool(api_key=api_keys['google_api_key'])]

In [5]:
llm=OpenAIModel('gpt-4.1-mini',provider=OpenAIProvider(api_key=api_keys['openai_api_key']))

In [6]:
agent=TAgent(llm=llm, voice=False)

In [7]:
result=await agent.chat(['what can you do'])
result

Initializing MCP server connection...
Connecting to MCP server (this may take a moment on first run)...


AgentRunResult(output='Hi! I can assist you with a variety of tasks such as answering questions, providing information, helping with writing and editing, brainstorming ideas, offering explanations on many topics, doing calculations, setting reminders, and more. If there’s something specific you want help with, just let me know! How can I assist you today?')

In [ ]:

llms={'pydantic_llm':GoogleModel('gemini-2.5-flash', provider=GoogleProvider(api_key=api_keys['google_api_key']))}